In [1]:
# 서울버스정류장
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)

# 2021.09.20일 수집 완료
query = ("select seoulsttsName from public_seoulbusstts ps where 6022 <= seq and seq <= 7022")

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
now = datetime.now()
nowDate = now.strftime('%Y%m%d')

cnt = 1
for i in location_list:
    seoulsttsName = i
    
    characters = "(',')"

    join_data = ''.join( x for x in seoulsttsName if x not in characters)
    print("조회날짜 : {} | 순번 : {} | start {} public_seoulbusstts".format(nowDate, cnt, join_data))
    url = 'http://ws.bus.go.kr/api/rest/stationinfo/getStationByName'
    queryParams = '?' + urlencode({
        quote_plus('serviceKey') : public_serviceKey(),
        quote_plus('stSrch') : join_data
    })

    response = requests.get(url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('itemList')
    resultCode = xmlobj.findAll('headerCd')

    if resultCode[0].text == '7':
        print("{} 번째부터 조회수 제한이 걸렸습니다.".format(seoulsttsName))
        mydb.close()
        break
    cnt += 1
    for j in rows:
        item = j
        arsId = item.findAll('arsId')
        stId = item.findAll('stId')
        stNm = item.findAll('stNm')
        lat = item.findAll('tmY')
        lng = item.findAll('tmX')
        
        try:
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
        except IndexError:
            print(item)
            sql = "INSERT INTO public_seoulbusarsid_info (arsId, stId, stNm, lat, lng) VALUES (?,?,?,?,?)"
            val = (arsId[0].text, stId[0].text, stNm[0].text, lat[0].text, lng[0].text)
            
        cursor.execute(sql, val)
        mydb.commit()

조회날짜 : 20210920 | 순번 : 1 | start 능안로입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 2 | start 굴레방다리 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 3 | start 육교앞 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 4 | start 광염교회 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 5 | start 풍림1차아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 6 | start 스위스그랜드호텔 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 7 | start 웨딩타운 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 8 | start 서대문문화체육회관 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 9 | start 서대문사거리 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 10 | start 경기대사회교육원 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 11 | start 문화촌현대아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 12 | start 팔각정 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 13 | start 남양아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 14 | start 이대전철역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 15 | start 서대문전철역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 16 | start 인창중고 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 17 | start 홍은유원아

조회날짜 : 20210920 | 순번 : 136 | start 유진상가 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 137 | start 인왕산현대아파트101동 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 138 | start 홍은2동주민센터 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 139 | start 뜨란채아파트101동 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 140 | start 뜨란채아파트101동 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 141 | start 홍지문35통 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 142 | start 옥천교앞 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 143 | start 대명빌라 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 144 | start 인왕시장 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 145 | start 창덕에버빌 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 146 | start 영천동 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 147 | start 독립문 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 148 | start 영천동 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 149 | start 동원베네스트아파트입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 150 | start 홍은유원아파트입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 151 | start 홍은사회복지관별관 public_seoulbusstts

조회날짜 : 20210920 | 순번 : 266 | start 신촌로터리 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 267 | start 서강대학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 268 | start 서강대학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 269 | start 광성중고등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 270 | start 광성중고등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 271 | start 대흥역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 272 | start 대흥역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 273 | start 용강초등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 274 | start 동도중학교.서울디자인고등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 275 | start 동도중학교.서울디자인고등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 276 | start 공덕역2번출구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 277 | start 공덕역1번출구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 278 | start 서서울농협홍대역점 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 279 | start 동교로사거리 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 280 | start 성산동 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 281 | start 성산2교 public_seoulbusst

조회날짜 : 20210920 | 순번 : 396 | start 하늘초등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 397 | start 월드컵파크10단지 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 398 | start 월드컵파크11.12단지 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 399 | start 수색교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 400 | start 수색교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 401 | start 가좌역1번출구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 402 | start 가좌역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 403 | start 디지털미디어시티역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 404 | start 청구아파트.참누리아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 405 | start 청구아파트.참누리아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 406 | start 중동현대아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 407 | start 중동현대아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 408 | start 가좌행복주택.건영월드컵아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 409 | start 가좌행복주택.건영월드컵아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 410 | start 월드컵아이파크 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 411 | star

조회날짜 : 20210920 | 순번 : 529 | start 상암고등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 530 | start 상암월드컵파크67단지 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 531 | start 상암월드컵파크67단지 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 532 | start 상암고등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 533 | start 월드컵파크6단지상가 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 534 | start 공덕역10번출구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 535 | start 공덕역10번출구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 536 | start 웨딩타운 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 537 | start 상지초등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 538 | start 아현전철역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 539 | start 스탠포드호텔 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 540 | start 우리기술사옥.한국지역정보개발원 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 541 | start 전자회관 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 542 | start 상암중학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 543 | start 월드컵파크6단지 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 544 | start YTN뉴스퀘어.CJE

조회날짜 : 20210920 | 순번 : 662 | start 경의선숲길공원입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 663 | start 우성아파트입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 664 | start 성미약수터 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 665 | start 홍대입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 666 | start 공항철도.홍대입구역 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 667 | start 경성중고후문 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 668 | start 우성아파트3동 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 669 | start 하모니마트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 670 | start 성모병원사거리 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 671 | start 코오롱아파트사거리 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 672 | start 도화현대1차아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 673 | start 마포한강아이파크아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 674 | start 중동초등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 675 | start 신한은행서교동금융센터점 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 676 | start 경성중고.홍익디자인고 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 677 | start 망원1동파출소

조회날짜 : 20210920 | 순번 : 792 | start 신정2동주민센터 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 793 | start 오목교역.동신한방병원 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 794 | start 신목초등학교입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 795 | start 목동14단지상가 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 796 | start 현대아파트정문 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 797 | start 현대아파트정문 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 798 | start 삼성아파트정문 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 799 | start 삼성아파트후문 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 800 | start 현대아파트후문 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 801 | start 목동14단지C상가앞 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 802 | start 목일중학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 803 | start 목동13단지 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 804 | start 목동14단지 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 805 | start 신트리공원 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 806 | start 목동12.13단지사잇길 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 807 | start 목동12.13단지사잇

조회날짜 : 20210920 | 순번 : 918 | start 양강중학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 919 | start 신월2동주민센터입구.경창전통시장 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 920 | start 강서초등학교 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 921 | start 한빛종합사회복지관 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 922 | start 과학수사연구소입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 923 | start 서부트럭터미널.신정동일하이빌아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 924 | start 신정현대아파트.서부트럭터미널 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 925 | start 넓은들공원 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 926 | start 넓은들공원 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 927 | start 신정현대아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 928 | start 신정숲속마을아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 929 | start 신정숲속마을아파트.댓골마을입구 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 930 | start 푸른마을2단지아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 931 | start 푸른마을2단지아파트 public_seoulbusstts
조회날짜 : 20210920 | 순번 : 932 | start 신월시영아파트.강월초등학교 public_seoulbusstts
조회날짜 